# In-Class Assignment: Multiple Regression
# Day 14
# CMSE 202

### <p style="text-align: right;"> &#9989; Put your name here</p>
#### <p style="text-align: right;"> &#9989; Put your group member names here</p>

---
### Goals

By the end of today's class, you'll have practiced:
* Loading and manipulated fixed width column data
* Replacing/removing missing data entries
* Performing multiple regression using all features and a reduced set of statistically significant features.

### Agenda for today's class:

1. [Working with more unfamiliar data](#data)
1. [Multiple Regression](#multiple-regression)
1. [Visualization - How well does our model fit our data?](#viz)

### Imports

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set_context("notebook")
import pandas as pd
import statsmodels.api as sm


---
<a id="data"></a>
## 1. Working with more unfamiliar data 

We are going to work with some data generated by U.N.E.S.C.O. (United Nations Education, Scientific, and Cultural Organization) and data they collected relating to poverty and inequality in the world. There are two files you need to do the work:

- `poverty.dat` which is the data file itself
- `poverty.txt` which describes the data columns as **fixed width column** data. That is, this file describes the columns of the data for each category. For example, the data in columns 1-6 of `poverty.dat` contain the "live birth rates per 1,000 population".

You can download the files from here:

`https://raw.githubusercontent.com/msu-cmse-courses/cmse202-supplemental-data/main/data/poverty.dat`

`https://raw.githubusercontent.com/msu-cmse-courses/cmse202-supplemental-data/main/data/poverty.txt`

### How does one deal with a "fixed width column" data file?

Conveniently there is a fixed width column pandas data reader. Look it up and read in the data. **Check with your group members to make sure every can find the right function to use!**

Again we find ourselves with a data file does doesn't contain any column headers (argh!).  Take a look at the `poverty.txt` file for column information and give the columns in your Pandas DataFrame short, but useful names.

**&#9989; Do This:** Read the data into a DataFrame and display the `head()` of the DataFrame. Remember that you can set the column labels by setting the `.columns` attribute to a list with the appropriate column labels.

In [15]:
# put your code here

# put your code here
poverty_df= pd.read_csv("poverty.dat")
df_colspecs = pd.read_fwf("poverty.dat", colspecs=[(0, 5), (6, 13), (14, 21), (22, 29), (30, 37), (38, 45), (46,51), (52, 73)],
                         names=['live birth rate', 'death rate', 'infant death', 'male life expectancy', 'female life expectancy', 'gross national product', 'country group', 'country'])

names=df_colspecs.head()
print(names.dtypes)
print(names)



live birth rate           float64
death rate                float64
infant death              float64
male life expectancy      float64
female life expectancy    float64
gross national product    float64
country group             float64
country                    object
dtype: object
   live birth rate  death rate  infant death  male life expectancy  \
0             24.0         5.0          30.0                  69.0   
1             12.0        11.0          14.0                  68.0   
2             13.0        11.0          11.0                  71.0   
3              1.0        12.0           7.0                  69.0   
4             11.0        13.0          14.0                  65.0   

   female life expectancy  gross national product  country group  \
0                    75.0                    60.0            NaN   
1                    74.0                   225.0            NaN   
2                    77.0                   298.0            NaN   
3                    

### 1.1 Examining the "type" of the data

**&#9989; Questions:**  Now look at the `.dtypes` of your DataFrame and comment on anything that doesn't immediately make sense to you. Do all of the columns have a type that matches your expectations? If not, what is it about the values in the DataFrame that is causing this?

<font size=+3>&#9998;</font> Do this - Erase this and put your answer here.

### 1.2 Handling missing data - Imputation

Let's face it, sometimes data is bad. Values are not recorded, or are mis-recorded, or are so far outside of your expectations that you suspect that there is something wrong. On the other hand, just **changing** the data seems like cheating. We have to work with what we have, and if we have to make changes it would be good to do that programmatically so that it is recorded for others to see. 

The process of <a href="https://en.wikipedia.org/wiki/Imputation_(statistics)"> imputation </a> is the statistical replacement of missing/bad data with substitute values.

**It turns out that we have a case of missing data in our dataset!** In the **Gross National Product (GNP)** column some of the values are set to " \* " indicating missing data. When Pandas reads in the column the only type that makes sense when both characters _and_ numbers are present is a string. Therefore Pandas chose to set the type to `object` instead of the expected `int64` or `float64`.

#### Using `numpy.nan` as a replacement

For better or worse, pandas assumes that "bad values" will be marked in the data as **NaN** which it can then represent using [NumPy's "`nan`"](https://numpy.org/doc/stable/reference/constants.html#numpy.nan). NaN is short for "Not a Number".

**If we can mark the missing data with "NaN" instead of "\*", we will have access to some of the imputation methods, which would allow us to replacing the NaN values with various substitution values (e.g. mean, median, specific value, etc.)**. 

There are (at least) **two ways** to do this:
1. You can do a `.replace` on the column using a dictionary of the form "{value to replace : new value, ...}". If you do this, **remember to save the result**. After you do this, you'll still need to change the column type from "object" to a "float64" in order to assure that the values are numeric values. Note that you cannot convert a `np.nan` to an integer but you can to a float.
2. You can convert the everything that can be converted to a number using the Pandas `.to_numeric()` function. Conveniently if you use the "`errors`" argument in the function you can force Pandas to convert any non-numbers to "`np.nan`" values. As with the previous method, you need to save the converted column in place of the column with the "\*" entries. This option has the benefit of not requiring an additional step of having to manually change the data type!

**&#9989; Do This:** Convert the missing entries in the GNP column to `np.nan` values and show the head of your modified DataFrame to ensure that the "NaN" values are showing up. Also print the `dtypes` to show that the column has changed type.

In [ ]:
# put your code here


#### Changing np.nan values

Now that "bad values" are marked as `numpy.nan`, we can use the DataFrame method `fillna` to change those values. For example:

In [19]:
df_colspecs["gross national product"].fillna(0)

0      60.0
1     225.0
2     298.0
3       0.0
4     278.0
      ...  
92     22.0
93     11.0
94     22.0
95     42.0
96     64.0
Name: gross national product, Length: 97, dtype: float64

The above cell returns a new DataFrame where all the `np.nan` values in the GNP column are replaced with 0.

You can do other things are well, for example:

In [29]:
# Two ways of accomplishing the same thing:

# Change the values in the series object (the column) directly
poverty_df["gross national product"].fillna(poverty_df["gross national product"].mean())

# Changes the value of the series object (the column) by accessing the full dataframe and using a dictionary to reference the column
poverty_df.fillna({"gross national product": poverty_df["gross national product"].mean()})

KeyError: 'gross national product'

**Both of the lines in the above cell do the same thing**. The first version changes any `np.nan` in the `GNP` column to be the mean of the column. The second takes a dictionary where the the key of the dictionary is the column to change and the value is what to replace the `np.nan` with. Note you could replace with other values like: median, min, max, or some other fixed value.

Remember that all of these examples return either a new Series (when working with just a column) or a DataFrame (if working with the entire element). Nothing is changed in the original unless you assign the result or use `inplace=True` in the call.

Finally, if you decide that the right thing to do is **remove** any row with a `np.nan` value, we can use the `.dropna` method of DataFrames as shown below:

In [32]:
len(poverty_df)
poverty_df_dropped = poverty_df.dropna()
print(len(poverty_df), len(poverty_df_dropped))

96 96


#### What do you think?

**&#9989; Do This:** Discuss with your group what you think is the best thing to do with the "bad values" in the DataFrame given the discussion above. Make a collective decision and record it below. Once you've come to a decision, modify your dataset accordingly.

<font size=+3>&#9998;</font> Do this - Erase this and put your answer here.

---
<a id="multiple-regression"></a>
## 2. Multiple Regression

In the past, we have limited ourselves to using a single feature or independent variable to fit a line or, as in the pre-class, created additional features based on our original feature to fit a polynomial. However, **we can just as easily use all, or some combination of all, the features available our dataset** to make a OLS model. This is referred to as multiple regression (you can see a brief introduction [here](https://medium.com/swlh/understanding-multiple-linear-regression-e0a93327e960)). The question is, **is it a good idea to just use all the possible features available to make a model?**

**&#9989; Do This:** Discuss this idea with your group and record your answer below.

<font size=+3>&#9998;</font> Do this - Erase this and put your answer here.

### 2.1 Infant Mortality model

Using the U.N.E.S.C.O. data, we can make a model of "Infant Mortality" as the dependent variable against all the other available features. As a hint, an easy way to do this is the make the `sm.OLS` model with  "Infant Mortality" as the first argument (the dependent variable) and then the entire DataFrame where "Infant Mortality" is dropped as the second argument. **You should also drop the "Country" column as unique strings don't play well in basic linear models.**

**&#9989; Do This:** Make an OLS model that predicts "Infant Mortality" using the other variables (making sure to drop the "Country" column as well) and display the `.summary()` of that process. 

In [45]:
# put your code here
poverty_df=df_colspecs.drop(columns=['country','country group'])
print(poverty_df)
poverty_df = poverty_df.replace([np.inf, -np.inf], np.nan).dropna()

X = poverty_df[['live birth rate', 'death rate', 'male life expectancy', 'female life expectancy','gross national product']]
X = sm.add_constant(X)
y = poverty_df['infant death']
poverty_model = sm.OLS(y, X).fit()
print(poverty_model.summary())

    live birth rate  death rate  infant death  male life expectancy  \
0              24.0         5.0          30.0                  69.0   
1              12.0        11.0          14.0                  68.0   
2              13.0        11.0          11.0                  71.0   
3               1.0        12.0           7.0                  69.0   
4              11.0        13.0          14.0                  65.0   
..              ...         ...           ...                   ...   
92             52.0        15.0          10.0                  49.0   
93             50.0         1.0          10.0                  51.0   
94             45.0        14.0           8.0                  50.0   
95             51.0        13.0           8.0                  50.0   
96             41.0        10.0           6.0                  56.0   

    female life expectancy  gross national product  
0                     75.0                    60.0  
1                     74.0               

There are several interesting things about this `.summary()`. Let's start with things you have seen before.

**&#9989; Do This:** Look for the adjusted $R^2$ statistic. What does this adjusted $R^2$ tell you about how well your model fits your data?

<font size=+3>&#9998;</font> Do this - Erase this and put your answer here.

Now, let's look at something new, **the "P" values** associated with the features used in the model. [P values](https://en.wikipedia.org/wiki/P-value) are used widely in statisical testing to judge if a result is statistically significant. Those P values that are 0 (or typically less 0.05) indicate a feature that is "significant" in its ability to predict the dependent variable. Those larger than 0.05 are less significant. Of course, one should be cautious with relying solely on P-values as they can be [misused](https://en.wikipedia.org/wiki/Misuse_of_p-values) and [p-hacking](https://en.wikipedia.org/wiki/Data_dredging) (intentional or not) can lead to misleading results.

**&#9989; Do This:** With a healthy dose of caution in mind, review your P-values. The values you get will depend on what you did with your "bad values", but list below the top three "most significant" features and the overall Adjusted R-squared using all the features.

<font size=+3>&#9998;</font> Do this - Erase this and put your answer here.

### 2.2 A "reduced" model using only the "significant" features

Modeling data is as much a craft as it is a science. We often seek the simplest models that explain or data well because they are typically more interpretable, easier to explain, and provide the information on the main influences of the system we are studying. There are reasons we might want a more complex model to capture the details and the nuance of the system. But for the U.N.E.S.C.O. data that we have, we are likely able to capture most of the system using a smaller number of features. _These ideas are related to the pre-class modeling you did with increasingly higher powers of `x`._

**&#9989; Do This:** Redo the model with only the top three features you found above vs "Infant Mortality". Display the summary.

In [ ]:
# your code here

**&#9989; Do This:** Review this model and the one you constructed earlier in the notebook. Report how the Adjusted R-squared value changed from using only the top three vs using all the available features. How well does this reduced model appear to fit your data?

<font size=+3>&#9998;</font> Do this - Erase this and put your answer here.

---
<a id="viz"></a>
## 3. Visualization - How well does our model fit our data?

We have been checking how our models fit our data using both plots of the fitted values and the residuals. These plots are generated from the information stored in [various attributes of the OLS results object](https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.OLSResults.html). We will continue to use the top two plots from `.graphics.plot_regress_exog` to investigate our fits. But you could also construct the plots directly using the attributes of the OLS results object.

**Note that you will need one plot for each feature in the model as each figure is only produced for a given choice of feature.**

**&#9989; Do This:** Create three `.graphics.plot_regress_exog` figures, one for each of the features in your reduced model. Pay special attention to the top two plots: the fitted values figure and the residual plot.

In [ ]:
# put your code here


**&#9989; Questions:** Based on these figures, how well does it appear your reduced model fit your data? Do you have any concerns about the distribution of the residuals?

<font size=+3>&#9998;</font> Do this - Erase this and put your answer here.

---
<a id="viz"></a>
## 4. Symbolic Regression - Searching for both model structure and parameters!

So far we've been exploring scenarios where we have a model structure and the goal is to fit the model to some data by finding the parameter values that lead to the best fit. While this works great if we know the model structure, in cases where we don't know the model structure, we don't want to be left guessing and checking many different structure. Rather, we can let the computer automatically search for the best model structure for us. 

This combined search for both the model structure and parameters is called Symbolic Regression. 

In this section we will explore StackGP ([StackGP Documentation](https://hoolagans.github.io/StackGP-Documentation/)), a Python package for Symbolic Regression, to see how we can find a model that fits data when we don't know the correct model structure. 

**Note: this exercise is just for fun. You will not be tested on your knowledge of symbolic regression. Although, you may choose to use it in your course project if it makes sense.**


**&#9989; Do This:** First use pip install to install StackGP. `pip install StackGP` or `pip install --user StackGP`

**&#9989; Do This:** Make a scatter plot using the data in the code cell below. 

In [ ]:
import numpy as np
import StackGP as sgp
import matplotlib.pyplot as plt
x=np.array([[1. , 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2,
        2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5,
        3.6, 3.7, 3.8, 3.9, 4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8,
        4.9, 5. , 5.1, 5.2, 5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6. , 6.1,
        6.2, 6.3, 6.4, 6.5, 6.6, 6.7, 6.8, 6.9, 7. , 7.1, 7.2, 7.3, 7.4,
        7.5, 7.6, 7.7, 7.8, 7.9, 8. , 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7,
        8.8, 8.9, 9. , 9.1, 9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9]])
y=np.array([42.44135808, 41.20429358, 40.06103995, 39.0029947 , 38.02249497,
       37.11269916, 36.26748529, 35.48136346, 34.74940027, 34.06715341,
       33.43061493, 32.83616182, 32.28051306, 31.76069195, 31.27399325,
       30.81795425, 30.39032942, 29.98906797, 29.61229414, 29.25828969,
       28.92547844, 28.61241251, 28.31776005, 28.04029442, 27.77888439,
       27.53248546, 27.30013212, 27.08093081, 26.87405365, 26.67873284,
       26.49425552, 26.31995922, 26.15522768, 25.99948711, 25.8522028 ,
       25.712876  , 25.58104116, 25.45626332, 25.33813586, 25.22627828,
       25.12033436, 25.01997034, 24.92487328, 24.83474962, 24.74932377,
       24.66833691, 24.59154578, 24.51872164, 24.44964933, 24.38412631,
       24.32196185, 24.26297627, 24.2070002 , 24.15387398, 24.10344696,
       24.055577  , 24.01012991, 23.96697899, 23.92600454, 23.88709345,
       23.8501388 , 23.81503951, 23.78169997, 23.75002973, 23.71994321,
       23.6913594 , 23.66420161, 23.6383972 , 23.61387741, 23.59057706,
       23.56843443, 23.54739099, 23.52739129, 23.50838274, 23.49031551,
       23.47314232, 23.45681832, 23.44130099, 23.42654998, 23.41252699,
       23.3991957 , 23.38652164, 23.37447206, 23.36301591, 23.3521237 ,
       23.34176742, 23.33192048, 23.32255764, 23.31365491, 23.30518953])



**&#9989; Question:** Based on the scatter plot of the data, do you have a guess as to what the model structure might be? Make your best guess and write out the equation below. 

<font size=+3>&#9998;</font> Do this - Erase this and put your answer here.

**&#9989; Do This:** Now remake the scatter plot from above, but add in a line or points representing the model from your guess to see how close you got with your guess. 

**&#9989; Do This:** Now lets use Symbolic Regression to search for a model. Just run the code below and the search will begin. (Note: this may take a few seconds since it has to search a lot of model structures). You should see a plot showing the learning curve of the search after it completes.  

In [ ]:
model=sgp.evolve(x,y,tracking=True)[0]

**&#9989; Do This:** Now run the following code to see the equation that was discovered by the search. 

In [ ]:
sgp.printGPModel(model)

**&#9989; Do This:** Now take a look at the documentation for `plotModelResponseComparison`. Use the plotting function to see how well the model fits the data. 

**&#9989; Do This:** Run the following code. This will display the ground truth equation that was used to generate the training data. Did the model search find a similar model? 

In [ ]:
truthModel=[np.array([sgp.exp, sgp.sqrt, sgp.inv, 'pop', sgp.add, sgp.exp], dtype=object),
 [sgp.variableSelect(0),
  3.141592653589793],
 [np.float64(0.0), 9]]
sgp.printGPModel(truthModel)

**&#9989; Do This:** Now lets try fitting the data from part 2 where the country column was dropped. Note: the data frame with the input features will need to be transposed since the `evolve` function expects each feature to be a row. You can easily transpose a data frame by doing something like `dfT = df.T`

**&#9989; Do This:** The following code will take the model you evolved and display it using the variable names from the dataset. 

In [ ]:
from sympy import symbols
sgp.printGPModel(model,symbols(["BirthRate", 
        "DeathRate",  
        "LifeExpectancyM", 
        "LifeExpectancyF", 
        "GNP", 
        "Group"]))

-----
### Congratulations, you're done with your in-class assignment!

Now, you just need to submit this assignment by uploading it to the course <a href="https://d2l.msu.edu/">Desire2Learn</a> web page for today's submission folder (Don't forget to add your names in the first cell).

##### &#169; Copyright 2025,  Department of Computational Mathematics, Science and Engineering at Michigan State University

<!-- 9/16/2024 -->